In [3]:
import numpy as np
import os
from typing import Dict, Tuple

import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras

import keras.layers as l
from keras import models, callbacks, utils, losses

In [21]:
text = ''
with open('garri-potter-i-metody-racionalnogo-myshleniya.txt', 'r', encoding='utf_8_sig') as file:
    text = file.read(250000)

def get_features_target(seq: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 100

alphabet = np.array(sorted(set(text)))

word_index = {char: i for i, char in enumerate(alphabet)}
index_word = {i: char for i, char in enumerate(alphabet)}

sequences = Dataset.from_tensor_slices(np.array([word_index[char] for char in text])).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

In [22]:
model = keras.Sequential([
    l.Embedding(len(alphabet), BATCH_SIZE, batch_input_shape=[BATCH_SIZE, None]),
    l.SimpleRNN(512, return_sequences=True, stateful=True),
    l.SimpleRNN(512, return_sequences=True, stateful=True),
    l.Dense(len(alphabet))
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=25, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/25
25/25 [==============================] - 12s 430ms/step - loss: 3.5733 - accuracy: 0.1271
Epoch 2/25
25/25 [==============================] - 10s 406ms/step - loss: 3.3351 - accuracy: 0.1475
Epoch 3/25
25/25 [==============================] - 10s 415ms/step - loss: 3.3282 - accuracy: 0.1475
Epoch 4/25
25/25 [==============================] - 11s 418ms/step - loss: 3.2977 - accuracy: 0.1511
Epoch 5/25
25/25 [==============================] - 11s 457ms/step - loss: 3.1121 - accuracy: 0.1994
Epoch 6/25
25/25 [==============================] - 11s 421ms/step - loss: 2.8642 - accuracy: 0.2208
Epoch 7/25
25/25 [==============================] - 11s 439ms/step - loss: 2.7284 - accuracy: 0.2422
Epoch 8/25
25/25 [==============================] - 10s 419ms/step - loss: 2.6423 - accuracy: 0.2621
Epoch 9/25
25/25 [==============================] - 11s 432ms/step - loss: 2.6135 - accuracy: 0.2757
Epoch 10/25
25/25 [==============================] - 10s 414ms/step - loss: 2.4970 - accura

In [45]:
def predict(sample, model, tokenizer, vocabulary, n_next, temperature, batch_size, word = False):
    if word:
        sample_vector = [tokenizer[word] for word in sample.split()]
    else:
        sample_vector = [tokenizer[char] for char in sample]
    predicted = sample_vector
    sample_tensor = tf.expand_dims(sample_vector, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    for i in range(n_next):
        pred = model(sample_tensor)
        pred = pred[0].numpy() / temperature
        pred = tf.random.categorical(pred, num_samples=1)[-1, 0].numpy()
        predicted.append(pred)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([pred], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    pred_seq = [vocabulary[i] for i in predicted]
    generated = ' '.join(pred_seq) if word else ''.join(pred_seq)
    return generated

In [30]:
print(predict(
    sample='Хороший',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=150,
    temperature=0.5,
    batch_size=BATCH_SIZE
))

Хороший надел Гарри стала он не стало приходил на подобным вери подобном, который голоси всехо «Поттер собренного приверения и последней и слова то ты тогда 


In [34]:
print(predict(
    sample='Гермиона',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=100,
    temperature=0.25,
    batch_size=BATCH_SIZE
))

Гермиона подобные волшебником с тобой слова с тобой могут то положение проверить тебя в своей случает способ


In [36]:
print(predict(
    sample='Волшебник',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=125,
    temperature=0.8,
    batch_size=BATCH_SIZE
))

Волшебником зарозней от отванил и постростить.

— Поторую шилопрюдённой с приверили расле Сиберци зачется в Лодим сезедное, но понныва


In [48]:
print(predict(
    sample='Гарри',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=200,
    temperature=0.4,
    batch_size=BATCH_SIZE
))

Гарри Поттер не потому тоберь в оборожным они было способный рузом собрадным слове.

Гарри подобнилась в устольной тординулась в посторожение вольший тебя в посмотрел на сображение проверила и последном об
